In [ ]:
import pandas as pd
import icalendar
import requests
from io import BytesIO
from datetime import datetime

In [ ]:
# Function to download and parse ICS file
def parse_ics(url):
    response = requests.get(url)
    calendar = icalendar.Calendar.from_ical(response.content)
    events = []
    for component in calendar.walk():
        if component.name == "VEVENT":
            event = {
                'summary': component.get('summary'),
                'dtstart': component.get('dtstart').dt,
                'dtend': component.get('dtend').dt,
                'location': component.get('location')
            }
            events.append(event)
    return events

# Highlight the games
def highlight_games(row):
    if pd.notna(row['location_8u']) and pd.notna(row['location_11u']):
        if row['location_8u'] != row['location_11u']:
            return ['background-color: yellow'] * len(row)
        else:
            return ['background-color: lightgreen'] * len(row)
    return [''] * len(row)

In [ ]:
# URLs for the schedules
url_8u = "webcal://api.team-manager.gc.com/ics-calendar-documents/user/ccbe7020-5bd2-4e1c-a765-279cfdcd3aef.ics?teamId=ed885410-3f79-44a3-bf00-e56aa53943a8&token=568f8c48fd9fdea20eab815a5321217fcf55744ede76cf3e3b6ddfba9b473d64"
url_11u = "webcal://api.team-manager.gc.com/ics-calendar-documents/user/ccbe7020-5bd2-4e1c-a765-279cfdcd3aef.ics?teamId=d682ed0f-9cb3-487f-99f2-8b95110d7747&token=361e72106b7e4e67c755cc497b71f561e7941223c849856f209bc56e9564373d"

# Parse the schedules
schedule_8u = parse_ics(url_8u)
schedule_11u = parse_ics(url_11u)

# Convert to DataFrame
df_8u = pd.DataFrame(schedule_8u)
df_11u = pd.DataFrame(schedule_11u)

# Merge the schedules on the date and time
merged_df = pd.merge(df_8u, df_11u, on='dtstart', suffixes=('_8u', '_11u'), how='outer')

# Apply the highlighting
styled_df = merged_df.style.apply(highlight_games, axis=1)

# Display the table
styled_df